# This shows planar waves, and their location in kspace

Distance from center = frequency\
Angle from horizontal =  tilt \
Complex portion = phase 

In [6]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib
import csv
import scipy.optimize as op
import scipy.stats
from scipy.optimize import curve_fit
import scipy.io
from scipy.stats import rice

import random
import pickle
import seaborn as sns

import numpy as np
from scipy.stats import ttest_ind, ttest_ind_from_stats, wilcoxon,ttest_rel, pearsonr
from scipy.special import stdtr
import csv
import pandas as pd

import sys 
import os

import matplotlib.image as mpimg


from tkinter import *
from tkinter import ttk


Using matplotlib backend: MacOSX


# plot 2d planar wave

In [7]:
fig, (ax1, ax2) = plt.subplots(1,2)

x = np.arange(0, 101, 1)
X, Y = np.meshgrid(x, x)
img = np.sin(2 * np.pi * X /20) #20 is the wavelength (how many pixels in one lil wave)
ax1.imshow(img,cmap = 'gray')
ax2.plot(img[51])
plt.show()

#so this has wavelength of 30
# pixel size of 3

# get FFT

In [8]:
fig, (ax1, ax2) = plt.subplots(1,2)
ft = np.fft.fft2(img)
kspace = np.fft.fftshift(ft) #moves zero frequency component to center
ax1.imshow(np.abs(kspace),cmap = 'gray')
ax2.plot(np.abs(kspace)[50])
plt.show()

arr = np.abs(kspace)
np.amax(np.abs(kspace))
np.where(arr==np.amax(arr))

(array([50, 50]), array([45, 55]))

# get iFFT

In [9]:
fig, (ax1,ax2,ax3,ax4) = plt.subplots(1,4)

ax1.imshow(img,cmap = 'gray')
ax2.imshow(np.abs(kspace),cmap = 'gray')

#ifft of k-space back into spatial domain
ift = np.fft.ifftshift(kspace)
iimg = np.fft.ifft2(ift)

#fft of ifft 
rekspace = np.fft.fft2(iimg)
rekspace = np.fft.fftshift(rekspace) #moves zero frequency component to center


ax3.imshow(np.real(iimg),cmap = 'gray')
ax4.imshow(np.abs(rekspace),cmap = 'gray')

plt.show()

# now that that is all set, lets rotate the sinusoid and show how that leads to rotating points in k-space

In [10]:
fig, (ax1, ax2) = plt.subplots(1,2)
angles = np.linspace(0,np.pi,10)
for tilt in angles:
    #phi being the tilt angle
    img = np.sin(2*np.pi*(X*np.cos(tilt) + Y*np.sin(tilt)) / 20)
    ax1.imshow(img,cmap = 'gray')
    ft = np.fft.fft2(img)
    kspace = np.fft.fftshift(ft) #moves zero frequency component to center
    ax2.imshow(np.abs(kspace),cmap = 'gray')

    plt.pause(.5)
    
    

[50 50] [45 55]
[48 52] [45 55]
[47 53] [46 54]
[46 54] [47 53]
[45 55] [49 51]
[45 55] [51 49]
[46 54] [53 47]
[47 53] [54 46]
[48 52] [55 45]
[50 50] [45 55]


# now include phase and show how that phase isn't ky (it's imaginary!)

In [143]:
fig, (ax1, ax2, ax3) = plt.subplots(1,3)
phases = np.linspace(0,np.pi,10)
for phi in phases:
    #phi being the tilt angle
    img = np.sin(2*np.pi*(X*np.cos(np.pi/4) + Y*np.sin(np.pi/4)) / 20 + phi)
    ax1.imshow(img,cmap = 'gray')
    ft = np.fft.fft2(img)
    kspace = np.fft.fftshift(ft) #moves zero frequency component to center
    ax2.imshow(np.abs(kspace),cmap = 'gray')
    ax3.imshow(np.imag(kspace),cmap = 'gray')
    
    
    plt.colorbar
    plt.pause(.5)

# now be able to get one of those points and reconstruct planar wave (approximation) from it


In [144]:
fig, (ax1, ax2, ax3, ax4) = plt.subplots(1,4)

ax1.imshow(img,cmap = 'gray')
ax2.imshow(np.abs(kspace),cmap = 'gray')

#find max
arr = np.abs(kspace)
np.amax(np.abs(kspace))
maxx,maxy = np.where(arr==np.amax(arr))

#get mask of a point (in this case it's the max)
mask = np.zeros(np.shape(kspace))
mask[maxx[0],maxy[0]] = 1
mask[maxx[1],maxy[1]] = 1
ax3.imshow(mask)

#now getting ifft of JUST ONE POINT in kspace
kspace_point = kspace*mask

ift = np.fft.ifftshift(kspace_point)
iimg = np.fft.ifft2(ift)
ax4.imshow(np.real(iimg),cmap = 'gray')

# YAY IT WORKS. okay so now be able to reconstruct an image from this... 